In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import json

In [2]:
# function courtesy of CryptoDataDownload.com
def fetch_daily_data(symbol):
    pair_split = symbol.split('/')  # symbol must be in format XXX/XXX ie. BTC/EUR
    symbol = pair_split[0] + '-' + pair_split[1]
    url = f'https://api.pro.coinbase.com/products/{symbol}/candles?granularity=86400'
    response = requests.get(url)
    if response.status_code == 200:  # check to make sure the response from server is good
        data = pd.DataFrame(json.loads(response.text), columns=['unix', 'low', 'high', 'open', 'close', 'volume'])
        data['date'] = pd.to_datetime(data['unix'], unit='s')  # convert to a readable date
        data['vol_fiat'] = data['volume'] * data['close']      # multiply the BTC volume by closing price to approximate fiat volume
                            
        # if we failed to get any data, print an error...otherwise write the file
        if data is None:
            print("Did not return any data from Coinbase for this symbol")
        else:
            data.to_csv(f'cb_{pair_split[0] + pair_split[1]}_daily.csv', index=False)
    else:
        print("Did not receieve OK response from Coinbase API")

In [3]:
fetch_daily_data('BTC/USD')

In [4]:
btc = pd.read_csv('cb_BTCUSD_daily.csv', index_col='date', parse_dates=[0])

In [5]:
btc.head()

,unix,low,high,open,close,volume,vol_fiat
date,,,,,,,
2021-01-11,1610323200,30100.00,38273.88,38168.89,31333.79,84503.239340,2.647807e+09
2021-01-10,1610236800,34444.00,41452.12,40257.43,38171.57,43736.570316,1.669494e+09
2021-01-09,1610150400,38800.00,41406.94,40642.15,40257.43,27152.971029,1.093109e+09
2021-01-08,1610064000,36565.08,41986.37,39510.55,40665.15,48522.484903,1.973174e+09
2021-01-07,1609977600,36200.00,40425.00,36859.26,39505.56,50346.305691,1.988959e+09


In [6]:
btc.drop('unix', axis=1, inplace=True)

In [7]:
btc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 300 entries, 2021-01-11 to 2020-03-18
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   low       300 non-null    float64
 1   high      300 non-null    float64
 2   open      300 non-null    float64
 3   close     300 non-null    float64
 4   volume    300 non-null    float64
 5   vol_fiat  300 non-null    float64
dtypes: float64(6)
memory usage: 16.4+ KB


In [8]:
btc.index = pd.to_datetime(btc.index)

In [9]:
btc.head()

,low,high,open,close,volume,vol_fiat
date,,,,,,
2021-01-11,30100.00,38273.88,38168.89,31333.79,84503.239340,2.647807e+09
2021-01-10,34444.00,41452.12,40257.43,38171.57,43736.570316,1.669494e+09
2021-01-09,38800.00,41406.94,40642.15,40257.43,27152.971029,1.093109e+09
2021-01-08,36565.08,41986.37,39510.55,40665.15,48522.484903,1.973174e+09
2021-01-07,36200.00,40425.00,36859.26,39505.56,50346.305691,1.988959e+09


In [10]:
btc.head()

,low,high,open,close,volume,vol_fiat
date,,,,,,
2021-01-11,30100.00,38273.88,38168.89,31333.79,84503.239340,2.647807e+09
2021-01-10,34444.00,41452.12,40257.43,38171.57,43736.570316,1.669494e+09
2021-01-09,38800.00,41406.94,40642.15,40257.43,27152.971029,1.093109e+09
2021-01-08,36565.08,41986.37,39510.55,40665.15,48522.484903,1.973174e+09
2021-01-07,36200.00,40425.00,36859.26,39505.56,50346.305691,1.988959e+09


In [11]:
btc = btc.sort_values('date')

In [12]:
def make_features(data):
    data['month'] = data.index.month
    data['dayofweek'] = data.index.dayofweek
    data['range'] = data['high'] - data['low']
    data['week_low_mean'] = data['low'].shift().rolling(7).mean()
    data['week_high_mean'] = data['high'].shift().rolling(7).mean()
    data['month_low_mean'] = data['low'].shift().rolling(30).mean()
    data['month_high_mean'] = data['high'].shift().rolling(30).mean()

In [13]:
make_features(btc)

In [14]:
btc.tail(10)

,low,high,open,close,volume,vol_fiat,month,dayofweek,range,week_low_mean,week_high_mean,month_low_mean,month_high_mean
date,,,,,,,,,,,,,
2021-01-02,29039.00,33300.00,29413.29,32225.91,46675.246521,1.504152e+09,1,5,4261.00,26599.172857,28301.687143,21593.519000,22838.525333
2021-01-03,32008.62,34810.00,32222.88,33080.66,36951.716506,1.222387e+09,1,6,2801.38,27249.028571,29227.115714,21931.985667,23294.337333
2021-01-04,27678.00,33666.99,33082.84,32019.99,46045.389685,1.474373e+09,1,0,5988.99,28139.895714,30144.687143,22379.738000,23803.122000
2021-01-05,29891.13,34499.67,32020.22,34030.64,42282.569200,1.438903e+09,1,1,4608.54,28367.297143,31028.971429,22685.638000,24285.689000
2021-01-06,33352.54,37000.00,34043.91,36859.26,45744.103200,1.686094e+09,1,2,3647.46,28946.925714,32043.671429,23052.742333,24788.003000
2021-01-07,36200.00,40425.00,36859.26,39505.56,50346.305691,1.988959e+09,1,3,4225.00,29809.898571,33183.920000,23534.460333,25373.584000
2021-01-08,36565.08,41986.37,39510.55,40665.15,48522.484903,1.973174e+09,1,4,5421.29,30981.327143,34770.077143,24134.460333,26077.767000
2021-01-09,38800.00,41406.94,40642.15,40257.43,27152.971029,1.093109e+09,1,5,2606.94,32104.910000,36526.861429,24765.329667,26855.735000
2021-01-10,34444.00,41452.12,40257.43,38171.57,43736.570316,1.669494e+09,1,6,7008.12,33499.338571,37684.995714,25461.829667,27617.419000
